In [2]:
import json
import pandas as pd
from radgraph import utils
import numpy as np 

### RadGraph

Original NER model based on radiology reports, requires >Python 3.8, <Python 3.10

Outputs dictionary with entities and relations described, broken down semantically to denote negation, severity etc. 



In [ ]:
from radgraph import RadGraph
# initialize the radgraph instance 
test_radgraph = RadGraph()
test_annotations = test_radgraph(["no evidence of acute cardiopulmonary process moderate hiatal hernia"])
print(test_annotations)

model_type not provided, defaulting to radgraph-xl


In [13]:
# Load the structured report database and designate column for impression and findings (sections radgraph is designed to annotate) 
df_path = '../BERTopic_Intensity/structured_reports.csv'
database = pd.read_csv(df_path)
database.head()
database['radgraph_text'] = database['findings'] + database['impression']
database

,study_id,subject_id,report_path,full_text,examination,indication,technique,comparison,findings,impression,has_comparison,report_length,radgraph_text
0,57106576,18110461,../data/files/p18/p18110461/s57106576.txt,FINAL REPORT EXAMINATION: CHEST (PA AND LAT) I...,chest (pa and lat),history: [REMOVED]f with cough,chest pa and lateral,[REMOVED],cardiac silhouette size is normal. mediastinal...,findings concerning for multifocal pneumonia. ...,True,645,cardiac silhouette size is normal. mediastinal...
1,52444794,15447063,../data/files/p15/p15447063/s52444794.txt,FINAL REPORT INDICATION: History of left-sided...,NaN,history of left-sided weakness for 12 hours. p...,ap and lateral radiographs of the chest.,NaN,mild cardiomegaly has been stable compared to ...,interval increase in consolidation at the left...,False,806,mild cardiomegaly has been stable compared to ...
2,58791719,13243522,../data/files/p13/p13243522/s58791719.txt,FINAL REPORT EXAMINATION: Chest radiograph IND...,chest radiograph,[REMOVED]m with cystic fibrosis and fever/coug...,frontal and lateral view.,comparison is made to multiple chest radiograp...,right chest wall port-a-cath ends at the cavoa...,chronic changes of cystic fibrosis as describe...,True,1136,right chest wall port-a-cath ends at the cavoa...
3,51779043,11423061,../data/files/p11/p11423061/s51779043.txt,FINAL REPORT HISTORY: Chest pain. TECHNIQUE: U...,NaN,NaN,upright ap view of the chest.,[REMOVED].,low lung volumes limit assessment of the lung ...,low lung volumes limit assessment of the lung ...,True,685,low lung volumes limit assessment of the lung ...
4,58785779,15379716,../data/files/p15/p15379716/s58785779.txt,"FINAL REPORT INDICATION: Pneumonia, cough, and...",NaN,"pneumonia, cough, and shortness of breath.",NaN,chest radiograph from [REMOVED] and chest ct f...,NaN,stable lingular and increased right middle lob...,True,600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2754,58255867,14236258,../data/files/p14/p14236258/s58255867.txt,FINAL REPORT INDICATION: [REMOVED]M with cough...,NaN,[REMOVED]m with cough // r/o pna,ap and lateral views of the chest.,[REMOVED]. [REMOVED] and ct torso from [REMOVED].,vague opacity projecting over the right mid/lo...,"vague right mid/lower opacity, nonspecific the...",True,695,vague opacity projecting over the right mid/lo...
2755,59289169,10623647,../data/files/p10/p10623647/s59289169.txt,WET READ: [REMOVED] [REMOVED] [REMOVED] 4:26 P...,chest: frontal and lateral views,history: [REMOVED]m with hypoxia // pna?,chest: frontal and lateral,[REMOVED],bilateral patchy pulmonary opacities appear sl...,patchy bilateral mid to lower lung opacities a...,True,1760,bilateral patchy pulmonary opacities appear sl...
2756,55182265,15116068,../data/files/p15/p15116068/s55182265.txt,WET READ: [REMOVED] [REMOVED] [REMOVED] 11:56 ...,chest radiograph.,"[REMOVED]f with cough, fever, sob // ? pna",single portable semi upright radiograph the ch...,"chest ct: [REMOVED], [REMOVED]",extensive bronchiectasis is again noted in the...,"extensive bilateral bronchiectasis, with super...",True,1100,extensive bronchiectasis is again noted in the...
2757,50696726,17025867,../data/files/p17/p17025867/s50696726.txt,FINAL REPORT INDICATION: Cough and weakness. C...,NaN,cough and weakness.,NaN,radiograph available from [REMOVED]. frontal a...,NaN,1. increase in density of a right lower and mi...,True,649,NaN


In [31]:
import pandas as pd
from tqdm import tqdm

pneumonia_radgraph = RadGraph()  # Initialize RadGraph

# Ensure column is properly formatted
database["radgraph_text"] = database["radgraph_text"].astype(str).str.strip()  # Ensure strings

# Apply RadGraph to each report individually (with progress bar)
tqdm.pandas(desc="Processing RadGraph")
database["extracts"] = database["radgraph_text"].progress_apply(pneumonia_radgraph)

# Verify results
print(database["extracts"].head())

model_type not provided, defaulting to radgraph-xl


Processing RadGraph:   0%|          | 0/2759 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing RadGraph:   0%|          | 4/2759 [00:00<04:20, 10.59it/s]/Users/JanayeCheong/Documents/DATATHON/nusdatathon2024/venv_310/lib/python3.10/site-packages/radgraph/dygie/data/dataset_readers/dygie.py:192: UserWarning: Document 0 has a sentence with a single token or no tokens. This may break the modeling code.
  warnings.warn(msg)
Processing RadGraph: 100%|██████████| 2759/2759 [04:05<00:00, 11.23it/s]

0    {'0': {'text': 'cardiac silhouette size is nor...
1    {'0': {'text': 'mild cardiomegaly has been sta...
2    {'0': {'text': 'right chest wall port - a - ca...
3    {'0': {'text': 'low lung volumes limit assessm...
4    {'0': {'text': 'nan', 'entities': {}, 'data_so...
Name: extracts, dtype: object


In [33]:
database.to_csv('ner_database.csv')

In [35]:
import torch
torch.save(database, 'ner_database.pt')